# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# useful library
import pandas as pd
import numpy as np
import random
from math import sqrt
import matplotlib.pyplot as plt

# torch stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 00 Game Info

In [ ]:
STATE_SIZE = (3,3)
N_ACTIONS = STATE_SIZE[0]*STATE_SIZE[1]
STATE_DIM = 3 # first player 정보 넣음
BOARD_SHAPE = (STATE_DIM, 3, 3)

# 01 HYPER PARAMS
#### PV_EVALUATE_COUNT: MCTS 성능을 향상
#### EVAL_COUNT: 평가의 신뢰도 향상

In [ ]:
# 신경망 구조 관련
N_KERNEL = 64 # 합성곱 레이어(ConvLayer)의 필터 수 / 클수록 더 많은 패턴을 학습
N_RESIDUAL_BLOCK = 16 # Residual Block 개수 / 클수록 더 깊은 네트워크를 만들 수 있음

# 학습 관련
BATCH_SIZE = 128 # 한 번의 학습에서 처리하는 데이터 샘플 개수
EPOCHS = 50 # 전체 학습 반복 횟수 -> 늘려보자!
LEARNING_RATE = 0.001 # 학습률 (뉴런 가중치 업데이트 속도)
LEARN_EPOCH = 100 # 학습률 감소 주기

# Self-play 관련 -> 한번 selfplay 돌린 후 학습
SP_GAME_COUNT = 20  # self-play에서 생성할 게임 수
SP_TEMPERATURE = 1.5  # self-play에서 탐색 다양성을 조절하는 온도 파라미터

# 평가 관련
EVAL_COUNT = 100  # 평가할 때 MCTS 탐색을 몇 번 수행할지 결정
PV_EVALUATE_COUNT = 800 # MCTS가 얼마나 깊이 탐색할지를 결정 -> 400으로 수정
EVAL_GAME_COUNT = 20  # 평가 1회 당 게임 수(오리지널: 400)
EVAL_TEMPERATURE = 1.0  # 온도 파라미터

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Env

In [ ]:
class Environment: #틱택토 게임 환경 정의 클래스 / 게임 규칙을 코드로 구현한 것
    def __init__(self):
        self.n = STATE_SIZE[0] #보드 행 또는 열 크기
        self.num_actions = self.n ** 2
        self.action_space = np.arange(self.num_actions)
        self.reward_dict = {'win': 1, 'lose': -1, 'draw': 0} #결과에 따른 보상 정의
        #승리 1점 / 패배 -1점 / 무승부 0점


    def step(self, present_state, action_idx): #현재 상태에서 주어진 행동에 따라 게임 진행 / 행동 결과를 계산
        """
        present_state에 대해 action_idx의 행동에 따라 게임을 한 턴 진행시키고
        next_state, is_done, is_lose를 반환한다.
        """
        #action_idx: 플레이어가 선택한 행동의 인덱스 / 보드 칸
        #현재 상태에서 행동하는 행동을 수행하여 다음 상태를 계산
        next_state = present_state.next(action_idx)
        is_done, is_lose = next_state.check_done()
        #next_state에서 게임이 종료되었는지 확인
        #현재 플레이어가 패배했는지 확인

        return next_state, is_done, is_lose


    def get_reward(self, final_state): #게임 종료 후 보상 계산 / 승패 및 무승부 판정
        """
        게임이 종료된 state에 대해 각 플레이어의 reward를 반환한다.
        final_state: 게임이 종료된 state
        note: final_state가 is_lose라면, 해당 state에서 행동할 차례였던 플레이어가 패배한 것.
        """
        '''
        장예원 수정!
        '''
        is_done, is_lose = final_state.check_done()  # 게임 종료 및 패배 여부 확인

        # 승리, 패배, 무승부 정확하게 판별
        if not is_done:
            return 0  # 게임이 아직 끝나지 않았으면 보상을 주지 않음

        # ✅ 먼저, 무승부인지 확인
        if not is_lose and final_state.total_pieces_count() == self.num_actions:
            return self.reward_dict['draw']  # 무승부 시 0

        # ✅ 승패 보상을 반영하되, 선공/후공이 동일한 기준을 갖도록 설정
        reward = self.reward_dict['lose'] if is_lose else self.reward_dict['win']

        # ✅ 현재 state의 플레이어가 선공인지 후공인지에 따라 보상 반전
        return reward if final_state.check_first_player() else -reward


    def render(self, state): #게임 상태 출력
        '''
        입력받은 state를 문자열로 출력한다.
        X: first_player, O: second_player
        '''
        is_first_player = state.check_first_player() #현재 차례가 첫번째 플레이어인지 확인
        board = state.state - state.enemy_state if is_first_player else state.enemy_state - state.state
        board = board.reshape(3,3) #현재 보드 상태를 정리

        board_list = [['X' if cell == 1 else 'O' if cell == -1 else '.' for cell in row] for row in board]
        #돌 상태를 문자로 변환 -> 읽기 쉽게 표시
        #X: 첫번째 플레이어의 돌 / O: 두번째 플레이어의 돌 / .: 빈칸
        formatted_board = "\n".join([" ".join(row) for row in board_list])
        #보기 좋게 줄바꿈하여 출력
        return formatted_board  #print() 대신 문자열 반환

In [ ]:
import copy

In [ ]:
class State(Environment):
    def __init__(self, state=None, enemy_state=None, move_count=0):
        super().__init__()
        self.state = np.array(state if state is not None else np.zeros(STATE_SIZE, dtype=int))
        self.enemy_state = np.array(enemy_state if enemy_state is not None else np.zeros(STATE_SIZE, dtype=int))
        self.move_count = move_count  # ✅ move_count를 인자로 받아 명확하게 설정


    def total_pieces_count(self):
        '''
        이 state의 전체 돌의 개수를 반환한다.
        '''
        total_state = self.state + self.enemy_state
        return np.sum(total_state)


    def get_legal_actions(self):
        '''
        이 state에서 가능한 action을
        one-hot encoding 형식의 array로 반환한다.
        '''
        total_state = (self.state + self.enemy_state).reshape(-1)
        legal_actions = np.array([total_state[x] == 0 for x in self.action_space], dtype = int)
        return legal_actions


    def check_done(self):
        '''
        이 state의 done, lose 여부를 반환한다.
        note: 상대가 행동한 후, 자신의 행동을 하기 전 이 state를 확인한다.
        따라서 이전 state에서 상대의 행동으로 상대가 이긴 경우는 이 state의 플레이어가 진 경우이다.
        '''
        is_done, is_lose = False, False

        # Check lose
        lose_condition = np.concatenate([
            self.enemy_state.sum(axis=0),
            self.enemy_state.sum(axis=1),
            [np.trace(self.enemy_state)],  # 왼쪽 위 → 오른쪽 아래 대각선 검사
            [np.trace(np.fliplr(self.enemy_state))]  # 오른쪽 위 → 왼쪽 아래 대각선 검사
        ])

        if np.any(lose_condition == self.n):  # 확실히 3이 있는지 체크
            is_done, is_lose = True, True

        # Check draw (승리 조건이 없을 경우만 무승부 확인)
        if not is_lose and self.total_pieces_count() == self.n ** 2:
            is_done, is_lose = True, False

        # 디버깅 출력
        #print(f"\n[DEBUG] check_done() 호출됨")
        #print(f"[DEBUG] Current Board:\n{self.state - self.enemy_state}")  # 현재 보드 상태 출력
        #print(f"[DEBUG] Lose condition: {lose_condition}")  # lose_condition 값 출력
        #print(f"[DEBUG] is_done: {is_done}, is_lose: {is_lose}")  # 종료 여부 출력

        return is_done, is_lose


    def next(self, action_idx):
        '''
        주어진 action에 따라 다음 state를 생성한다.
        note: 다음 state는 상대의 차례이므로 state 순서를 바꾼다.
        '''
        x, y =np.divmod(action_idx, self.n)
        state = self.state.copy()
        state[x, y] = 1
        enemy_state = self.enemy_state.copy()

        # ✅ move_count를 정확히 1 증가
        new_move_count = self.move_count + 1

        next_state = State(enemy_state, state, new_move_count)
        return next_state


    def check_first_player(self):
        # State()를 기본적으로 생성하면 move_count = 0이므로 check_first_player()는 항상 True
        return self.move_count % 2 == 0


    def get_random_action(self):
        '''
        이 state에서 가능한 action 중 랜덤으로 action을 반환한다.
        '''
        legal_actions = self.get_legal_actions()
        legal_action_idxs = np.where(legal_actions != 0)[0]
        action = np.random.choice(legal_action_idxs)
        return action


    def __str__(self):
        return Environment().render(self)  # state를 인자로 명확히 전달

# 03 Network

## Dual Network

### ConvLayer (보드 공간 정보를 유지하면서 승리 / 방어 조건을 나타내는 패턴을 자동으로 감지)

In [ ]:
class ConvLayer(nn.Module):
    def __init__(self, state_dim, n_kernel, kernel_size=3, padding=1, dropout_rate=0.0):
        #입력 채널 수
        #출력 채널 수
        #필터 크기 (3X3)
        #패딩: 입력과 출력 크기 동일하게 유지
        #바이어스 항 생략 (배치 정규화 사용 시 바이어스 항을 생략하는 것이 일반적)
        #드롭아웃 비율
        super().__init__()
        self.conv = nn.Conv2d(3, n_kernel, kernel_size=kernel_size, padding=padding, bias=False)
        self.bn = nn.BatchNorm2d(n_kernel) #배치 정규화 사용할 채널 수
        self.relu = nn.ReLU(inplace=True)  # in-place 연산으로 메모리 효율성 증가
        self.dropout = nn.Dropout2d(dropout_rate) if dropout_rate > 0 else None

    def forward(self, x):
        x = self.conv(x) #입력 데이터에 합성곱 연산을 적용하여 특징맵 생성
        x = self.bn(x) #배치 정규화 적용
        x = self.relu(x) #비선형 활성화 함수 적용
        if self.dropout:  # 드롭아웃 적용 (필요시)
            x = self.dropout(x)
        return x

### Residual Block
##### 1. 배치 정규화 (학습 안정성)와 ReLU 활성화 함수 (비선형성)를 사용 -> 일반 CNN 동일
##### 2. 잔차 연결을 추가하여 기존 CNN 구조를 확장 -> 출력값이 아닌 잔차를 학습
##### 3. 깊은 신경망에서도 기울기 소실 문제를 완화 -> 일반 CNN 문제점 완화

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, n_kernel, kernel_size=3, padding=1, dropout_rate=0.0):
        super().__init__()
        self.conv1 = nn.Conv2d(n_kernel, n_kernel, kernel_size=kernel_size, padding=padding, bias=False)
        #합성곱 연산 수행
        #residual block은 입력과 출력 채널이 동일해야 한다는 제약을 따른다~
        self.bn1 = nn.BatchNorm2d(n_kernel) #배치 정규화 적용
        self.conv2 = nn.Conv2d(n_kernel, n_kernel, kernel_size=kernel_size, padding=padding, bias=False)
        self.bn2 = nn.BatchNorm2d(n_kernel)
        self.relu = nn.ReLU(inplace=True)  # 활성화 함수 객체화
        self.dropout = nn.Dropout2d(dropout_rate) if dropout_rate > 0 else None


    def forward(self, x):
        residual = x #입력을 잔차로 저장

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x) # double check
        if self.dropout:  # 드롭아웃 적용
            x = self.dropout(x)
        x = self.conv2(x)
        x = self.bn2(x)

        x += residual
        #모델이 잔차를 학습하도록 하여 기울기 소실 문제를 완화
        x = self.relu(x)

        return x

### Network -> Alphazero 스타일의 딥러닝 모델
#### 정책(Policy)과 값(Value)을 동시에 출력하는 신경망을 사용!
##### 1. 정책망: 현재 상태에서 가능한 행동들에 대해 확률 분포를 반환
##### 2. 가치망: v=1 내가 반드시 승리할 상태 / v=-1 상대방이 반드시 승리할 상태

In [ ]:
class Network(nn.Module):
    def __init__(self, n_residual_block, n_kernel, input_shape, n_actions, dropout_rate=0.0):
        super().__init__()
        self.n_residual_block = n_residual_block #residual block를 쌓아 네트워크 깊이를 조정
        self.n_kernel = n_kernel #합성곱 레이어의 필터 수
        self.input_shape = input_shape #입력 데이터의 크기 ex)틱택토 (3X3X3)
        self.n_actions = n_actions #길이 계산 -> 가능한 행동의 총 개수를 저장 ex)틱택토 9개
        self.n_fc_nodes = n_kernel  # Match to the number of kernels

        # nn
        self.conv_layer = ConvLayer(self.input_shape[0], self.n_kernel) # input_shape[0] = 3
        self.residual_blocks = nn.Sequential() #모듈을 차례로 실행
        for i in range(self.n_residual_block):
            self.residual_blocks.add_module(f"residual_block_{i+1}", ResidualBlock(self.n_kernel))
        #각 residual block에 이름을 부여하여 추가
        self.global_pooling = nn.AdaptiveAvgPool2d(1)
        #공간 차원을 축소해 각 채널별로 하나의 평균값을 반환
        # heads and softmax ftn for policy
        self.policy_head = nn.Sequential(
            nn.Linear(self.n_fc_nodes, self.n_actions),
            nn.Softmax(dim=-1),
        )
        #fully connected layer를 통해 행동별 확률 분포를 계산 / 이 행동을 선택했을 때 유리할 가능성
        #self.n_fc_nodes: 각 출력 노드 값 (특정 행동 선택 확률) / self.n_actions: 보드 칸 수 (9)
        #p = [0.1, 0.3, 0.2, ..., 0.4]
        #policy head의 출력값을 확률 분포로 변환
        self.value_head = nn.Sequential(
            nn.Linear(self.n_fc_nodes, 1),
            nn.Tanh(),
            )
        #fully connected layer를 통해 현재 상태의 가치를 평가 / 현재 상태의 승리 가능성
        #v = 0.8
        #value head의 출력값을 [-1,1]로 제한

        self.dropout = nn.Dropout(dropout_rate) if dropout_rate > 0 else None


    def forward(self, x):
        x = self.conv_layer(x)
        x = self.residual_blocks(x)
        x = self.global_pooling(x) #conv -> residual blocks -> global pooling을 거쳐 1D로 변환
        x = x.view(x.size(0), -1)  # Flatten

        if self.dropout:
            x = self.dropout(x)  # Optional Dropout

        p = self.policy_head(x)  # 행동별 확률 분포
        v = self.value_head(x)  # 현재 상태의 가치 평가

        return p, v

## Test Code

In [ ]:
N_RESIDUAL_BLOCK = 16
N_KERNEL = 64
N_ACTIONS = 9
INPUT_SHAPE = (3, 3, 3)  # (채널, 높이, 너비)

model = Network(N_RESIDUAL_BLOCK, N_KERNEL, INPUT_SHAPE, N_ACTIONS, dropout_rate=0.1) #배치 크기, 채널 수, 높이, 너비
#알파제로의 정책 가치 네트워크

# example
x = torch.randn((2, STATE_DIM, 3, 3), requires_grad=True)

# policy and value
p, v = model(x) #입력 텐서를 Network 모델에 통과시켜 정책 p와 가치 v를 계산
p, v = p.detach().numpy(), v.detach().numpy()

print(f"정책의 출력 크기: {p.shape}")  # 예상 출력: (2, 9)
print(f"가치의 출력 크기: {v.shape}")  # 예상 출력: (2, 1)
print("정책 출력:\n", p)
print("가치 출력:\n", v)

정책의 출력 크기: (2, 9)
가치의 출력 크기: (2, 1)
정책 출력:
 [[0.12674648 0.08209809 0.49866658 0.06866132 0.08782072 0.0182383
  0.03088977 0.03405626 0.05282244]
 [0.03800518 0.04073203 0.5444247  0.20422314 0.08512492 0.01014396
  0.0198465  0.03819559 0.01930401]]
가치 출력:
 [[ 0.030595 ]
 [-0.6824331]]


# 04 MCTS

### Node (MCTS에서 하나의 상태를 표현하는 노드)
#### PUCT 알고리즘: Upper Confidence Bound for Trees
1. 게임의 현재 상태 state
2. 해당 상태에서 가능한 행동 child_nodes,
3. 각 행동의 평가 정보 n,w,p로 구성된다!

In [ ]:
class Node:
    def __init__(self, state, p):
        self.state = state #현재 상태
        self.p = p # prior prob / residual 네트워크에서 예측된 행동 확률
        self.n = 0 # n_visit / 노드 방문 횟수
        self.w = 0 # cum weight / 노드 평가 가치를 누적한 값
        self.child_nodes = [] #현재 가능한 모든 행동을 통해 생성된 자식 노드


    def evaluate_value(self, model, C_PUCT): #model 이용하여 현재 노드 가치 평가 / 자식 노드를 확장 or 선택
        #model -> 정책 및 가치 평가를 수행할 모델
        #return -> 현재 노드의 평가 가치
        is_done, is_lose = self.state.check_done()

        if is_done: #게임이 끝난 상태인지 확인 / 끝난 상태라면 가치 계산
            # judge current value / Use the reward system from the common environment
            value = self.state.get_reward(self.state)
            self.n += 1 #노드 업데이트
            self.w += value
            #print(f"[DEBUG] MCTS - Terminal State Reached. Value: {value}")
            return value

        # When child node does not exist 자식 노드가 없는 경우
        if len(self.child_nodes) == 0:
            policy, value = predict(model, self.state) # by using nn
            #residual 네트워크를 호출하여 정책 및 가치 예측

            # update 평가 결과를 부모 노드로 역전파하여 업데이트
            self.n += 1 #탐색 후 방문 횟수 업데이트
            self.w += value #탐색 후 누적 가치를 업데이트
            #print(f"[DEBUG] MCTS - Expanding Node. Policy: {policy}, Value: {value}")

            # expand 현재 상태에서 가능한 행동을 기반으로 자식 노드 확장
            legal_actions = np.where(self.state.get_legal_actions() != 0)[0]
            policy = np.maximum(policy, 1e-3)  # 정책 값이 0이 되지 않도록 최소값 설정
            policy /= np.sum(policy)  # 정규화

            for action, p in zip(legal_actions, policy):
                self.child_nodes.append(Node(self.state.next(action), p))
                #가능한 행동과, 이에 대응하는 확률을 사용하여 새로운 자식 노드 생성

            return value

        # When child node exist 자식 노드가 있는 경우
        else:
            value = - self.select_next_child_node(C_PUCT).evaluate_value(model, C_PUCT)
            #PUCT 알고리즘을 사용해 최적의 자식 노드 선택 / 반환된 값을 반전

            # update
            self.n += 1
            self.w += value

            return value


    def select_next_child_node(self, C_PUCT): #PUCT 알고리즘을 사용하여 최적의 자식 노드 선택
        total_visit = sum(get_n_child(self.child_nodes)) #모든 자식 노드의 총 방문 횟수
        values = []
        for node in self.child_nodes: #각 자식 노드에 대해~
            q_value = -(node.w / node.n) if node.n != 0 else 0 #현재 노드의 평균 가치
            #방문하지 않은 노드는 0
            node_value = q_value + C_PUCT * node.p * sqrt(total_visit) / (1 + node.n) # PUCT 알고리즘
            #탐험 용어: 방문 횟수가 적고, 사전 확률 p가 높은 자식 노드를 더 많이 탐험
            values.append(node_value) #PUCT 점수: q_value + node_value를 계산

        return self.child_nodes[np.argmax(values)]
        #가장 높은 PUCT 점수를 가진 자식 노드를 선택하여 반환

### def for MCTS

### get_n_child

In [ ]:
def get_n_child(child_nodes): #자식 노드 리스트에서 각 노드의 방문 횟수를 추출

    child_n = [] #각 노드의 방문 횟수를 담은 리스트

    for node in child_nodes:
        child_n.append(node.n)

    return child_n

### predict

In [ ]:
def predict(model, state): #주어진 상태에서 정책 확률과 가치를 계산
    # 디바이스 설정
    device = next(model.parameters()).device
    # NumPy 배열로 변환
    input_array = np.array([state.state, state.enemy_state, state.state - state.enemy_state], dtype=np.float32)
    input_tensor = torch.tensor(input_array).unsqueeze(0).to(device)  # (1, 3, 3, 3)

    model.eval() #모델 평가 모드로 설정
    with torch.no_grad():
        policy, value = model(input_tensor)
        policy = policy.cpu().numpy().reshape(-1)
        value = value.cpu().numpy().item()

    # Value 정규화 (승패를 더욱 명확하게 구분)
    value = np.tanh(value * 25)  # 기존 8에서 15으로 증가하여 승패 차이를 더 극대화

    # Debugging policy values
    #print(f"[DEBUG] Predict() - Raw Policy: {policy}, Value: {value}")

    # ✅ 정책 값 정규화 및 NaN 방지
    if np.isnan(policy).any() or np.isinf(policy).any() or np.sum(policy) == 0:
        print(f"[WARNING] NaN detected in predict(). Resetting to uniform policy.")
        policy = np.ones_like(policy) / len(policy)

    # take legal policy
    legal_actions = np.where(state.get_legal_actions() != 0)[0]

    # ✅ 정책 크기 정합성 확인 및 보정
    if policy.shape[0] != N_ACTIONS:
        print(f"[WARNING] Policy shape mismatch in predict(). Expected {N_ACTIONS}, got {policy.shape[0]}. Adjusting...")
        policy = np.ones(N_ACTIONS) / N_ACTIONS # 기본 균일 분포 설정

    policy = policy[legal_actions] if len(legal_actions) > 0 else np.ones_like(policy) / len(policy)

    # ✅ 최종 확률 정규화
    policy /= np.sum(policy)

    #모델에서 반환된 정책 확률 중 현재 상태에서 가능한 행동의 확률만 포함된 리스트

    # Debugging normalized policy
    #print(f"[DEBUG] Predict() - Legal Policy: {legal_policy}")

    return policy, value

## MCTS (최적의 행동을 선택)
#### 각 루트 노드에서 시뮬레이션을 실행하여 다음 액션에 대한 확률을 계산
#### 값을 높이면 MCTS가 더 깊이 탐색하므로 더 정확한 정책/가치 평가를 제공

In [ ]:
class MCTS:
    def __init__(self, initial_pv_evaluate_count=800, max_pv_evaluate_count=1500, growth_rate=1.02, initial_c_puct=2.0, min_c_puct=1.5, decay_rate=0.99):
        self.policy = None #MCTS를 통해 계산된 행동 확률 분포를 저장
        self.pv_evaluate_count = initial_pv_evaluate_count #MCTS가 현재 상태를 탐색하는 횟수를 정의
        self.MAX_PV_EVALUATE_COUNT = max_pv_evaluate_count  # 최대 탐색 횟수
        self.GROWTH_RATE = growth_rate  # 증가율
        self.C_PUCT = initial_c_puct  # 초기 C_PUCT 값
        self.MIN_C_PUCT = min_c_puct  # 최소 C_PUCT 값
        self.DECAY_RATE = decay_rate  # 감소율


    def update_c_puct(self):
        """ 학습이 진행될수록 C_PUCT 값을 점진적으로 감소 """
        self.C_PUCT = max(self.MIN_C_PUCT, self.C_PUCT * self.DECAY_RATE)
        print(f"[DEBUG] C_PUCT updated to {self.C_PUCT:.3f}")


    def update_pv_evaluate_count(self):
        """ 학습이 진행될수록 MCTS 시뮬레이션 횟수를 점진적으로 증가 """
        self.pv_evaluate_count = min(self.MAX_PV_EVALUATE_COUNT, int(self.pv_evaluate_count * self.GROWTH_RATE))
        print(f"[DEBUG] Updated pv_evaluate_count: {self.pv_evaluate_count}")


    def get_policy(self, state, model, temp): #현재 상태에서 최적의 행동 확률 분포 계산
        original_temp = temp  # 기존 temp 값 저장

        if state.move_count == 0:
            temp = max(2.5, original_temp)  # Move 0에서는 온도를 높여 더 균등한 분포 유지

        root_node = Node(state, p=1.0)

        # ✅ 후공(Player 2)의 탐색을 보다 깊게 수행
        adjust_c_puct = self.C_PUCT * (1.3 if not state.check_first_player() else 1.0)
        evaluate_count = self.pv_evaluate_count + (100 if not state.check_first_player() else 50)

        for _ in range(evaluate_count):
            root_node.evaluate_value(model, adjust_c_puct)

        childs_n = get_n_child(root_node.child_nodes)
        legal_actions = np.where(state.get_legal_actions() != 0)[0]

        # ✅ 정책 값 shape 정합성 체크 및 보정
        if len(childs_n) != len(legal_actions):
            print(f"[WARNING] Mismatch detected in get_policy(): legal_actions={len(legal_actions)}, childs_n={len(childs_n)}.")

            # Fix: `childs_n` 크기를 `legal_actions` 크기로 조정
            if len(legal_actions) > 0:
                childs_n = np.ones(len(legal_actions))  # 균일한 확률로 보정
            else:
                childs_n = np.ones(env.num_actions)  # 기본 크기(9)로 설정

        policy = self.boltzmann_dist(childs_n, temp)

        # ✅ NaN 값 방지
        if np.isnan(policy).any() or np.isinf(policy).any() or np.sum(policy) == 0:
            print(f"[WARNING] Invalid policy detected in get_policy(). Resetting to uniform distribution.")
            policy = np.ones(len(legal_actions)) / len(legal_actions)
        else:
            policy /= np.sum(policy)  # 정규화

        temp = original_temp  # Move 0 후에는 원래 temp 값으로 복구

        return policy


    def get_actions_of(self, model, temp): #MCTS 통해 계산된 정책을 기반으로 상태에서 행동을 선택
        def get_action(state):
            self.policy = self.get_policy(state, model, temp)
            #get_policy를 호출하여 현재 상태에서 탐색된 행동 확률 분포를 계산하고 저장
            legal_actions = np.where(state.get_legal_actions() != 0)[0]
            action = np.random.choice(legal_actions, p=self.policy)
            #가능한 행동 중 하나를 선택 / 선택 확률은 self.policy에 따라 결정
            #ex) 가능한 행동 [0,1,2] / 행동 확률 [0.2, 0.5, 0.3]
            return action
        return get_action


    def boltzmann_dist(self, x_lst, temp):
        x_lst = np.array(x_lst, dtype=np.float32) + 1e-8  # 작은 값 추가하여 0으로 나누는 것을 방지

        # ✅ NaN 방지: x_lst 값이 너무 크거나 작으면 클리핑 적용
        x_lst = np.clip(x_lst, -20, 20)  # log scale에서 안정적인 값 유지

        x_lst = np.exp(x_lst / temp)  # 지수 함수 적용

        # ✅ 정책 정규화 (합이 1이 되도록 보정)
        x_lst /= np.sum(x_lst) if np.sum(x_lst) > 0 else len(x_lst)

        # ✅ NaN 확인 및 대체
        if np.isnan(x_lst).any() or np.isinf(x_lst).any():
            print(f"[WARNING] NaN detected in boltzmann_dist, replacing with uniform distribution.")
            x_lst = np.ones_like(x_lst) / len(x_lst)

        return x_lst.tolist()

# 05 SelfPlay *환경 적용에 따라 수정한 코드

## SelfPlay (두 에이전트가 대전하여 데이터 생성)
### MCTS와 정책-가치 네트워크를 활용
#### 랜덤 행동 비율을 추가하여 초반 데이터 다양성을 증가!

In [ ]:
# Self-play 데이터 크기 제한 (최신 N개의 게임만 유지)
MAX_HISTORY_SIZE = 5000  # 최근 5000개의 게임 기록만 유지

In [ ]:
class SelfPlay:
    def __init__(self, model, temp, sp_game_count, pv_evaluate_count, temp_discount=0.999995):
        # model
        self.model = model #정책-가치 네트워크 모델

        # params
        self.sp_game_count = sp_game_count #self-play 게임 횟수

        # about temps
        self.temp = temp #초기 볼츠만 온도 파라미터
        self.temp_discount = 0.999995 #온도 감소율

        self.history = [] # selfPlay's yield / self-play 결과를 저장 (게임 상태, 정책, 가치)

        self.pv_evaluate_count = pv_evaluate_count
        self.mcts = MCTS(self.pv_evaluate_count) #MCTS에 전달

        self.env = Environment()

        self.FIRST_PLAYER_BIAS = 0.6  # Player 1이 선공할 확률을 60%로 설정


    def get_first_player_value(self, ended_state): #게임 종료 상태에서 첫번째 플레이어의 승패를 평가
        # Use the common environment's reward system
        reward = self.env.get_reward(ended_state)
        return reward if ended_state.check_first_player() else -reward


    def _single_play(self, i): #하나의 self-play 게임을 수행하여 데이터를 생성
        history = [] #게임 중 저장된 데이터
        # Player 1이 선공할 확률을 FIRST_PLAYER_BIAS로 조정
        first_player = random.random() < self.FIRST_PLAYER_BIAS
        # 초기 상태 설정
        #state = State() if first_player else State(state=np.zeros((self.env.n, self.env.n)),
                                               #enemy_state=np.zeros((self.env.n, self.env.n)))

        # ✅ State 객체 생성 시 move_count 명확히 설정
        if first_player:
            state = State(move_count=0)  # 선공 (Player 1)
        else:
            state = State(state=np.zeros(STATE_SIZE, dtype=int),
                          enemy_state=np.zeros(STATE_SIZE, dtype=int),
                          move_count=2)  # 후공 (Player 2), move_count=2로 설정

        legal_actions = np.where(state.get_legal_actions() != 0)[0] # 현재 상태에서 가능한 행동들을 가져옴

        # 첫 번째 수를 MCTS 기반으로 선택
        if len(legal_actions) > 0:
            policy = self.mcts.get_policy(state, self.model, self.temp)  # MCTS로 정책 계산

            # ✅ 정책 값에서 NaN 확인 및 정규화
            if np.isnan(policy).any() or np.sum(policy) == 0:
                print(f"[WARNING] NaN detected in policy at Game {i + 1}. Replacing with uniform distribution.")
                policy = np.ones_like(legal_actions) / len(legal_actions)

            action = np.random.choice(legal_actions, p=policy)  # MCTS 정책 기반 선택
            state = state.next(action)
            # 선공에 따라 move_count를 올바르게 설정
            #state.move_count = 0 if first_player else 1 # 첫 수를 둔 후 move_count를 조정하여 불일치 방지

            print(f"Game {i + 1}: {'Player 1' if first_player else 'Player 2'} starts, first move at {action}")

        # 디버깅 메시지 추가
        print(f"[DEBUG] Game {i + 1}: first_player={first_player}")# check_first_player()={state.check_first_player()}")

        move_count = 0

        # 게임 진행 (MCTS를 사용하여 수를 선택)
        while not state.check_done()[0]: #여기서는 state.is_done()이 true일 때 루프 종료
            move_count += 1
            policy = self.mcts.get_policy(state, self.model, self.temp)

            # ✅ 정책 값에서 NaN 확인 및 정규화
            if np.isnan(policy).any() or np.sum(policy) == 0:
                print(f"[WARNING] NaN detected in policy during move {move_count} of Game {i + 1}.")
                policy = np.ones_like(legal_actions) / len(legal_actions)

            legal_actions = np.where(state.get_legal_actions() != 0)[0]

            # Check if legal_actions is empty before random move selection
            if len(legal_actions) == 0:
                print(f"Game {i + 1}: No legal actions available, ending game.")
                break  # or handle it differently

            final_policy = np.zeros([self.env.num_actions]) # init value / 행동 확률 초기화
            final_policy[legal_actions] = policy
            #현재 상태에서 가능한 행동에 대해서만 확률을 채우기
            #전체 행동 공간에 대한 정책이 완성!
            #불가능한 행동의 확률은 0으로 유지, 가능한 행동의 확률은 MCTS 결과에 기반~
            history.append([state, final_policy, 0]) #데이터 저장 (보드 상태, 정책, 가치)
            #state_n.board: 게임의 n번째 상태
            #policy_n: 해당 상태에서의 정책(확률 분포)
            #None: 가치(아직 업데이트되지 않음)

            action = np.random.choice(legal_actions, p=policy)
            state = state.next(action)

        # 게임 종료 후 보상 계산
        final_reward = self.env.get_reward(state)
        player1_reward = final_reward if first_player else -final_reward
        player2_reward = -player1_reward

        # 게임 결과 출력
        game_result = "Draw" if final_reward == 0 else "Player 1 Wins" if player1_reward > 0 else "Player 2 Wins"
        print(f"Game {i + 1} Result: {game_result} (P1: {player1_reward}, P2: {player2_reward})")

        # 구분선 추가 (게임 종료 후)
        print("-" * 20)  # 가독성을 위한 구분선

        # 히스토리의 가치 업데이트 (각 턴마다 반전)
        value = player1_reward
        for j in range(len(history)): #가치를 게임 히스토리에 업데이트
            history[j][-1] = value #각 상태 history[i]의 마지막 요소에 가치를 기록
            #state.board, final_policy, None -> state_i.board, policy_i, value
            value = -value #각 턴마다 가치를 반전하여 저장
            #첫번째 플레이어의 가치와 두번째 플레이어의 가치는 반대이다!
            #즉 한 플레이어가 유리한 상태일수록 상대방은 불리한 상태가 된다~

        # discount temp / 온도 감소
        if i >= 30:
            self.temp = max(0.75, self.temp * self.temp_discount)
        #초기 30턴까지는 온도를 유지, 이후에는 temp_discount를 적용하여 점진적으로 감소

        return history #종료 상태의 가치(점수)를 히스토리에 기록


    def _self_play(self): #지정된 횟수만큼 self-play를 반복 수행
        # sp_game_count 번 만큼 single play 시행
        for i in range(self.sp_game_count): #지정된 게임 횟수만큼 반복
            single_history = self._single_play(i) #한 번의 게임 실행
            self.history.extend(single_history) #게임 결과를 self.history 전체 히스토리에 추가

            # 메모리 사용량을 줄이기 위해 오래된 데이터 삭제
            if len(self.history) > MAX_HISTORY_SIZE:
                self.history = self.history[-MAX_HISTORY_SIZE:]  # 최근 기록만 유지

            if (i+1) % (self.sp_game_count // 10) == 0: #게임 진행 상태를 10% 단위로 출력
                print(f"self play {i+1} / {self.sp_game_count}")


    def __call__(self, execute_self_play=False):
        """
        SelfPlay 객체를 호출 시 self-play 실행 여부를 제어.
        - execute_self_play: True이면 self-play 실행, False이면 실행하지 않음.
        """
        if execute_self_play:
            print("Self-play 실행 중...")
            self._self_play()
        else:
            print("Self-play 실행이 비활성화되었습니다.")


    def reset_history(self):
        """
        Self-play 데이터를 초기화합니다.
        """
        self.history = []
        print("Self-play 데이터가 초기화되었습니다.")


    def update_model(self, model):  # 모델 업데이트 메서드
        self.model.load_state_dict(model.module.state_dict())  # 주어진 모델의 가중치를 현재 모델에 복사

# 06 TrainNetwork

## TrainNetwork
### 알파제로 알고리즘의 정책-가치 네트워크를 학습
#### self-play로 생성된 데이터를 사용하여 정책-가치 네트워크를 업데이트
1. 샘플링된 데이터를 준비
2. 손실 함수 계산 (정책 및 가치 손실)
3. 역전파를 통해 네트워크 업데이트

In [ ]:
class TrainNetwork:
    def __init__(self, model, batch_size, learning_rate, learn_epoch, save_loss=True):
        # define
        self.losses = [] # elements : ( p_loss, v_loss, total_loss )
        self.batch_size = batch_size

        # model & device
        self.model = model
        self.device = next(model.parameters()).device #네트워크가 실행될 디바이스 (GPU/CPU)

        # learning rate
        self.learning_rate = learning_rate
        self.learn_decay = 0.1
        self.learn_epoch = learn_epoch #학습률 감소 주기

        # define loss ftn / 손실 함수
        self.mse_loss = F.mse_loss #가치 손실 계산
        #네트워크가 예측한 가치와 실제 가치 간의 차이를 최소화
        #가치는 연속형 데이터로 [-1,1] 범위의 실수 (회귀 문제)
        self.cross_entropy_loss = nn.CrossEntropyLoss() #정책 손실 계산
        #네트워크가 예측한 정책과 실제 정책 간의 차이를 최소화
        #정책은 확률 분포로 클래스 확률 값, 즉 확률의 합이 1 (분류 문제)

        # Optimizer & Scheduler
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate, eps=1e-4, weight_decay=1e-4) # L2 정규화 포함
        #가중치를 학습시키기 위해 사용
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=self.learn_epoch, gamma = self.learn_decay)
        #학습률을 learn_epoch 주기로 감소시켜 학습

        self.save_loss = save_loss  # 손실을 저장할지 여부
        self.loss_file = "train_losses.csv"

        # CSV 파일이 없으면 헤더 추가
        if self.save_loss and not os.path.exists(self.loss_file):
            with open(self.loss_file, "w", newline="") as f:
                writer = csv.writer(f)
                writer.writerow(["epoch", "p_loss", "v_loss", "total_loss"])


    def _train(self, history, epoch): #self-play에서 생성된 데이터를 사용해 네트워크를 학습
        # 샘플링
        # history에 배치 샘플링을 위한 충분한 요소가 있는지 확인
        num_samples = len(history) # history의 실제 크기 가져오기
        batch_size = min(self.batch_size, num_samples)  # history가 더 짧으면 더 작은 배치를 사용
        sample = random.choices(history, k=batch_size) #history에서 데이터를 무작위로 샘플링

        states, target_policies, target_values = zip(*sample)
        #샘플 데이터는 상태, 목표 정책, 목표 가치로 분리

        # `reward` 값이 비정상적으로 -1로만 채워졌는지 확인
        # print(f"[DEBUG] Target values before normalization: {target_values}")

        # NumPy 배열로 변환
        expanded_states = np.array([
            np.array([s.state, s.enemy_state, s.state - s.enemy_state], dtype=np.float32)
            for s in states
        ])  # (BATCH, 3, 3, 3)

        #샘플 데이터를 파이토치 텐서로 변환
        states = torch.tensor(expanded_states, dtype=torch.float32)  # (BATCH, STATE_DIM, 3, 3)
        target_policies = torch.tensor(np.array(target_policies), dtype=torch.float32).view(batch_size, -1)  # (BATCH, 9)
        target_values = torch.tensor(np.array(target_values), dtype=torch.float32).view(batch_size, -1)  # (BATCH, 1)

        target_values = (target_values - target_values.mean()) / (target_values.std() + 1e-6)  # 정규화

        # print(f"[DEBUG] Target values after normalization: {target_values}")

        # device에 올리기 / 데이터를 디바이스 (GPU/CPU)에 전송
        states, target_policies, target_values = states.to(self.device), target_policies.to(self.device), target_values.to(self.device)

        # 학습 실행
        self.model.train()
        policy, value = self.model(states) #정책-가치 네트워크 실행

        # 손실 계산
        p_loss = self.cross_entropy_loss(policy, target_policies) #정책 손실
        v_loss = 0.25 * self.mse_loss(value, target_values) #가치 손실
        total_loss = p_loss + 0.5 * v_loss #총 손실 / 기존보다 Value Loss 기여도를 낮춤

        # Logging losses / 손실 기록
        self.losses.append((p_loss.item(), v_loss.item(), total_loss.item()))

        # 10 에포크마다 CSV 파일에 저장
        if epoch % 10 == 0 and self.save_loss:
            with open(self.loss_file, "a", newline="") as f:
                writer = csv.writer(f)
                writer.writerows([(epoch, p, v, t) for p, v, t in self.losses])
            self.losses.clear()  # 저장 후 메모리에서 삭제하여 불필요한 메모리 사용 방지

        # Backpropagation / 역전파 및 업데이트
        self.optimizer.zero_grad()
        total_loss.backward()
        self.optimizer.step() #가중치 업데이트
        self.scheduler.step() #학습률 감소


    def __call__(self, history, epoch):
        self._train(history, epoch) #호출 시 내부적으로 _train 매서드 실행
        #train_network._train(history) 직접 호출 -> train_network(history) 함수처럼 호출


    def update_model(self, model): #self-play에 사용되는 네트워크 모델을 업데이트 / 최신 학습된 모델로 교체
        self.model.load_state_dict(model.module.state_dict())
        #현재 클래스가 사용하는 모델을 외부에서 제공된 새로운 모델의 가중치로 업데이트

# 07 EvalNetwork
### 학습된 최근 모델을 현재 최고 모델과 비교 평가하는 역할

In [ ]:
import os

In [ ]:
save_path = '/content/drive/My Drive/3-2/강화'
os.makedirs(save_path, exist_ok=True)  # 폴더 생성

In [ ]:
class EvalNetwork: #모델 성능 평가를 담당
    def __init__(self, best_model, eval_game_count, eval_temperature, eval_count, env):
        # eval info
        self.eval_game_count = eval_game_count #평가 시 실행할 게임 수
        self.eval_temperature = eval_temperature #볼츠만 분포 온도 파라미터

        # models
        self.best_model = best_model #현재 최고 모델
        self.recent_model = None #비교할 최신 모델 / 초기화 시에는 None
        #평가 후, 최신 모델이 더 성능이 좋으면 최고 모델로 업데이트~

        # MCTS
        self.mcts = MCTS(eval_count) #평가에 사용할 MCTS 탐색 개체

        # update / win
        self.updated = False #최근 모델이 최고 모델로 업데이트되었는지 여부
        self.win_rate = 0.0 #최근 모델의 평균 승률

        self.env = env  # AlphaZeroAgent에서 전달받도록 수정

    def _first_player_point(self, ended_state):
        reward = self.env.get_reward(ended_state)
        # 첫 번째 플레이어일 경우 그대로 반환, 두 번째 플레이어일 경우 반전
        return reward if ended_state.check_first_player() else -reward


    def _evaluate_network(self, recent_model): #최신 모델을 현재 최고 모델과 평가
        self.updated = False #초기화
        self.recent_model = recent_model

        # MCTS를 이용
        next_actions_recent = self.mcts.get_actions_of(self.recent_model, self.eval_temperature)
        #최신 모델의 행동 생성 함수 / 각 모델의 행동 확률 생성
        next_actions_best = self.mcts.get_actions_of(self.best_model, self.eval_temperature)
        #최고 모델의 행동 생성 함수
        next_actions = (next_actions_recent, next_actions_best)

        total_point = 0

        for i in range(self.eval_game_count): #지정된 게임 수만큼 평가 진행
            if i % 2 == 0: #짝수 게임: 최신 모델이 선공
                total_point += self._single_play(next_actions)
            else: #홀수 게임: 최신 모델이 후공
                total_point += 1 - self._single_play(list(reversed(next_actions)))

        average_point = total_point / self.eval_game_count #각 게임 결과를 누적하여 평균 점수 계산
        print('Average Point of Latest Model', average_point)

        if average_point >= 0.5:
            self._update_best_model() #평균 점수가 0.5 이상이면 최고 모델 업데이트


    def _single_play(self, next_actions): #단일 게임을 실행하여 첫번째 플레이어의 점수 반환
        state = State() #초기 상태 생성
        while not state.check_done()[0]:
            next_action = next_actions[0] if state.check_first_player() else next_actions[1]
            #행동 생성 함수를 호출하여 행동 선택
            #next_actions[0]: next_actions_recent
            #next_actions[1]: next_actions_best
            action = next_action(state)
            state = state.next(action) #선택된 행동에 따라 다음 상태로 이동
        return self._first_player_point(state) #첫번째 플레이어의 점수 반환


    def _update_best_model(self): #최신 모델을 현재 최고 모델로 업데이트
        #DataParallel로 감싸져 있는지 확인하고, 상태에 따라 적절히 state_dict를 로드
        if isinstance(self.recent_model, torch.nn.DataParallel):
            recent_model_state_dict = self.recent_model.module.state_dict()  # DataParallel 제거
        else:
            recent_model_state_dict = self.recent_model.state_dict()

        if isinstance(self.best_model, torch.nn.DataParallel):
            self.best_model.module.load_state_dict(recent_model_state_dict)  # DataParallel 제거 후 로드
        else:
            self.best_model.load_state_dict(recent_model_state_dict)

        self.updated = True
        print("Best Model is Updated.")


    #현재 최고 모델의 가중치를 지정된 경로에 저장
    def save_model(self, file_name="best_model_env_weights.pth"):
        save_file_path = os.path.join(save_path, file_name)
        if isinstance(self.best_model, torch.nn.DataParallel):
            torch.save(self.best_model.module.state_dict(), save_file_path)
        else:
            torch.save(self.best_model.state_dict(), save_file_path)
        print(f"Model saved to {save_file_path}")


    def __call__(self, recent_model):
        self._evaluate_network(recent_model)

# 08. AlphaZeroAgent

In [ ]:
import time

In [ ]:
# Updated main code with optimizations / device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class AlphaZeroAgent:
    def __init__(self, model_path, sp_game_count, eval_game_count, learning_rate, batch_size, epochs, learn_epoch, pv_evaluate_count, env):
        """
        AlphaZeroAgent 초기화 메서드
        """
        # 모델 초기화 및 가중치 로드
        self.model = Network(N_RESIDUAL_BLOCK, N_KERNEL, STATE_SIZE, N_ACTIONS)
        self.model = nn.DataParallel(self.model).to(device) #Multi-GPU 활용

        # 가중치 로드
        if model_path:
            params = torch.load(model_path, map_location=device)  # map_location 추가
            self.model.load_state_dict(params, strict=False)

        # SelfPlay, TrainNetwork, EvalNetwork 초기화
        self.selfplay = SelfPlay(self.model, SP_TEMPERATURE, sp_game_count, pv_evaluate_count)
        self.train_network = TrainNetwork(self.model, batch_size, learning_rate, LEARN_EPOCH)
        self.eval_network = EvalNetwork(self.model, eval_game_count, EVAL_TEMPERATURE, EVAL_COUNT, env)

        # 학습 관련 설정
        self.epochs = epochs
        self.sp_game_count = sp_game_count


    def get_action(self, state):
        """
        MCTS를 이용해 현재 상태에서 최적의 행동을 반환합니다.
        """
        if not hasattr(self, 'selfplay') or not hasattr(self.selfplay, 'mcts'):
            raise AttributeError("MCTS 객체가 초기화되지 않았습니다.")

        # MCTS를 사용하여 행동 확률을 계산
        action_probs = self.selfplay.mcts.get_policy(state, self.model, SP_TEMPERATURE)
        # 확률에 따라 행동을 선택
        legal_actions = np.where(state.get_legal_actions() != 0)[0]
        return np.random.choice(legal_actions, p=action_probs)


    def train(self):
        for i in range(self.epochs):
            print(f'\n[Epoch {i + 1}/{self.epochs}] --------------------------------')


            for j in range(self.sp_game_count):  # self-play 한 판 → 학습 한 판 반복
                print(f'\n[Game {j + 1}/{self.sp_game_count} in Epoch {i + 1}]')


                # Self-play 수행 (한 판만 진행)
                start_time = time.time()
                with torch.no_grad():
                    self.selfplay._single_play(j)  # 한 판만 수행
                print(f"Single Self-play 완료. 소요 시간: {time.time() - start_time:.2f}초")

                # Training 수행 (바로 학습)
                start_time = time.time()
                self.train_network(self.selfplay.history, i+1)  # 손실 반환
                print(f"Training 완료. 소요 시간: {time.time() - start_time:.2f}초")

                # Loss 출력
                if len(self.train_network.losses) > 0:
                    latest_p_loss, latest_v_loss, latest_total_loss = self.train_network.losses[-1]
                    print(f"🔹 Game {j + 1}: Policy Loss = {latest_p_loss:.4f}, Value Loss = {latest_v_loss:.4f}, Total Loss = {latest_total_loss:.4f}")

                # Model evaluation (한 epoch 단위로 평가)
                start_time = time.time()
                self.eval_network(self.train_network.model)
                print(f"Evaluation 완료. 소요 시간: {time.time() - start_time:.2f}초")

                # C_PUCT 및 MCTS 시뮬레이션 횟수 업데이트 적용
                self.selfplay.mcts.update_c_puct()
                self.selfplay.mcts.update_pv_evaluate_count()

                # 최신 모델이 최고 모델보다 성능이 좋다면 업데이트
                if self.eval_network.updated:
                    print(f"Saving best model at epoch {i + 1}...")
                    self.eval_network.save_model(f"model_env_checkpoint_epoch_{i + 1}.pth")  # 에포크별 파일 저장
                    self.eval_network.save_model()  # 기본 파일 이름(best_model_env_weights.pth)으로 저장
                    print("Model saved.")

                    print("최신 모델이 최고 모델로 업데이트되었습니다.")
                    self.eval_network.updated = False  # 업데이트 상태 초기화

        print("Training completed!")


# 09. Main
#### self-play, 모델 학습, 평가, 최고 모델 업데이트 -> 모델 성능 점진적으로 개선

In [ ]:
import csv

In [ ]:
if __name__ == '__main__':
    import torch

    # 필요한 상수 정의
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    N_KERNEL = 64
    N_RESIDUAL_BLOCK = 16

    BATCH_SIZE = 128
    EPOCHS = 50  # 학습 에포크
    LEARNING_RATE = 0.001
    LEARN_EPOCH = 100

    SP_GAME_COUNT = 20
    SP_TEMPERATURE = 1.5 # 볼츠만 분포의 온도 파라미터

    EVAL_COUNT = 100
    PV_EVALUATE_COUNT = 800
    EVAL_GAME_COUNT = 20
    EVAL_TEMPERATURE = 1.0

    # 이전 학습 가중치를 사용하지 않고 새 모델로 초기화
    model_path = None  # None으로 설정하거나 관련 인수를 제거

    # Environment 객체 생성
    env = Environment()

    # AlphaZeroAgent 초기화
    alphazero_agent = AlphaZeroAgent(
        model_path=None,  # 이전 가중치를 사용하지 않음
        sp_game_count=SP_GAME_COUNT,
        eval_game_count=EVAL_GAME_COUNT,
        learning_rate=LEARNING_RATE,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        learn_epoch=LEARN_EPOCH,
        pv_evaluate_count=PV_EVALUATE_COUNT,  # MCTS 탐색 깊이 증가
        env=env
    )

    # Self-play 실행을 명시적으로 제어
    alphazero_agent.selfplay(execute_self_play=True)  # Self-play 비활성화

    # 학습 실행
    alphazero_agent.train()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
--------------------
Single Self-play 완료. 소요 시간: 13.73초
Training 완료. 소요 시간: 0.26초
🔹 Game 20: Policy Loss = 2.0406, Value Loss = 0.0613, Total Loss = 2.0713
Average Point of Latest Model 0.45
Evaluation 완료. 소요 시간: 66.15초
[DEBUG] C_PUCT updated to 1.500
[DEBUG] Updated pv_evaluate_count: 1500

[Epoch 3/50] --------------------------------

[Game 1/20 in Epoch 3]
Game 1: Player 1 starts, first move at 1
[DEBUG] Game 1: first_player=True
Game 1 Result: Player 1 Wins (P1: 1, P2: -1)
--------------------
Single Self-play 완료. 소요 시간: 11.58초
Training 완료. 소요 시간: 0.44초
🔹 Game 1: Policy Loss = 2.0435, Value Loss = 0.0519, Total Loss = 2.0695
Average Point of Latest Model 0.5
Best Model is Updated.
Evaluation 완료. 소요 시간: 64.12초
[DEBUG] C_PUCT updated to 1.500
[DEBUG] Updated pv_evaluate_count: 1500
Saving best model at epoch 3...
Model saved to /content/drive/My Drive/3-2/강화/model_env_checkpoint_epoch_3.pth
Model saved to /content/drive/My Drive/3-2/강화/best_model_

KeyboardInterrupt: 

## Test Code
### 알파제로 모델과 사람이 틱택토 게임을 플레이하는 인터페이스를 제공
#### 모델 -> MCTS를 사용하여 행동을 선택
#### 사람 -> 키보드 입력으로 행동을 선택

In [ ]:
'''
class ModelvsHuman: #알파제로와 사람이 대결하는 코드
    def __init__(self, model):
        self.model = model #알파제로 모델 저장
        self.mcts = MCTS(EVAL_COUNT) #MCTS 객체 초기화하여 모델 기반의 행동 생성
        self.next_actions = self.mcts.get_actions_of(model, EVAL_TEMPERATURE)
        #MCTS가 생성한 행동 확률 분포를 사용하여 행동을 선택


    def render(self, state): #게임 현재 상태를 사람이 이해할 수 있는 형태로 변환하여 출력
        is_first_player = state.check_first_player()
        board = (
            state.state - state.enemy_state
            if is_first_player
            else state.enemy_state - state.state
        )
        #첫번째 플레이어와 두번째 플레이어의 보드를 합산하여 현재 보드 상태를 표현
        #각 플레이어의 돌을 구별하기 위해 값을 1 (첫번째 플레이어)와 -1 (두번째 플레이어)로 변환
        #첫번째 플레이어 차례일 때: state.board[0] + (-1 * state.board[1])
        #board = [[ 1, -1,  0],
        #         [ 0,  1,  0],
        #         [ 0,  0, -1]]
        #두번째 플레이어 차례일 때: state.board[1] + (-1 * state.board[0])
        #board = [[-1,  1,  0],
        #         [ 0, -1,  0],
        #         [ 0,  0,  1]]
        def pattern(x): #숫자 상태 (1,0,-1)을 'O','공백','X'로 변환
            if x == 1:
                return 'O' #플레이어 말 1 (첫번째 플레이어) -> 'O'
            elif x == 0:
                return ' ' #플레이어 말 0 -> '공백'
            else:
                return 'X' #플레이어 말 -1 (두번째 플레이어) -> 'X'

        board_list = list(map(pattern, board.reshape(-1)))
        # Fix: Convert sub-lists to strings before joining
        board_string = '\n'.join([' | '.join(board_list[i:i + 3]) for i in range(0, 9, 3)])
        print(board_string)


    def vs_human(self, with_policy=True): #알파제로 모델과 인간 간의 게임 진행
        """
        사람과 AlphaZero가 대국하며, 사람이 후공을 선택할 경우 직접 입력할 수 있도록 변경.
        """
        # 선공 / 후공 선택
        player_choice = input("선공(O)으로 플레이하려면 '1', 후공(X)으로 플레이하려면 '2'를 입력하세요: ").strip()
        while player_choice not in ["1", "2"]:
            player_choice = input("잘못된 입력입니다. '1'(선공) 또는 '2'(후공)을 입력하세요: ").strip()

        human_first = player_choice == "1"  # 사람이 선공인지 여부
        state = State()
        self.render(state)

        while not state.check_done()[0]:
           if state.check_first_player() == human_first:  # 사람이 둘 차례
                 legal_actions = np.where(state.get_legal_actions() != 0)[0]
                 my_action = int(input("Choose Your Action (0-8): "))
                 while my_action not in legal_actions:
                     print(f"Invalid action. Choose from {legal_actions}")
                     my_action = int(input("Choose Your Action (0-8): "))
                 state = state.next(my_action)
                 print(f"Human ({'O' if human_first else 'X'}) chose: {my_action}")
           else:  # 알파제로가 둘 차례
                 action = self.next_actions(state)
                 state = state.next(action)
                 print(f"AlphaZero ({'X' if human_first else 'O'}) chose: {action}")

           self.render(state)  # 현재 보드 상태 출력

        print("Game Over!")


    def __call__(self, with_policy=True): #기본 설정으로 알파제로 정책을 시각화하게 시킴
        self.vs_human(with_policy)
'''

In [ ]:
'''
class ModelvsHuman:
    def __init__(self, model):
        self.model = model
        self.mcts = MCTS(EVAL_COUNT)
        self.next_actions = self.mcts.get_actions_of(model, EVAL_TEMPERATURE)

    def render(self, state):
        board = state.state - state.enemy_state if state.check_first_player() else state.enemy_state - state.state
        board_list = [['O' if cell == 1 else 'X' if cell == -1 else ' ' for cell in row] for row in board]
        formatted_board = "\n".join([" | ".join(row) for row in board_list])
        print(formatted_board)

    def vs_human(self):
        """
        AlphaZero와 사람이 번갈아가며 정확히 교대하며 두도록 수정된 코드.
        """
        # 선공/후공 선택
        player_choice = input("선공(O)으로 플레이하려면 '1', 후공(X)으로 플레이하려면 '2'를 입력하세요: ").strip()
        while player_choice not in ["1", "2"]:
            player_choice = input("잘못된 입력입니다. '1'(선공) 또는 '2'(후공)을 입력하세요: ").strip()

        human_first = player_choice == "1"
        state = State()
        self.render(state)

        while not state.check_done()[0]:  # 게임이 종료되지 않았을 때
            if state.check_first_player() == human_first:  # 사람이 둘 차례
                legal_actions = np.where(state.get_legal_actions() != 0)[0]
                my_action = input(f"Choose Your Action {tuple(legal_actions)}: ").strip()

                # 유효한 입력값만 허용
                while not my_action.isdigit() or int(my_action) not in legal_actions:
                    my_action = input(f"잘못된 입력입니다. {tuple(legal_actions)} 중에서 선택하세요: ").strip()

                my_action = int(my_action)
                state = state.next(my_action)
                print(f"Human ({'O' if human_first else 'X'}) chose: {my_action}")

            else:  # AlphaZero 차례
                action = self.next_actions(state)
                state = state.next(action)
                print(f"AlphaZero ({'X' if human_first else 'O'}) chose: {action}")

            self.render(state)  # 현재 보드 상태 출력

        print("Game Over!")  # 게임 종료 메시지 출력

    def __call__(self):
        self.vs_human()
'''

In [ ]:
class ModelvsHuman:
    def __init__(self, model):
        self.model = model
        self.mcts = MCTS(EVAL_COUNT)
        self.get_action = self.mcts.get_actions_of(model, EVAL_TEMPERATURE)

    def render(self, state):
        """현재 게임 상태를 보기 좋게 출력"""
        board = state.state - state.enemy_state if state.check_first_player() else state.enemy_state - state.state
        board_list = [['O' if cell == 1 else 'X' if cell == -1 else ' ' for cell in row] for row in board]
        formatted_board = "\n".join([" | ".join(row) for row in board_list])
        print(formatted_board)
        print("-" * 11)  # 구분선 추가

    def vs_human(self):
        """AlphaZero와 사람이 정확히 번갈아가며 플레이하도록 수정"""
        # ✅ 선공/후공 선택
        player_choice = input("선공(O)으로 플레이하려면 '1', 후공(X)으로 플레이하려면 '2'를 입력하세요: ").strip()
        while player_choice not in ["1", "2"]:
            player_choice = input("잘못된 입력입니다. '1'(선공) 또는 '2'(후공)을 입력하세요: ").strip()

        human_first = player_choice == "1"
        state = State()
        self.render(state)

        while not state.check_done()[0]:  # 게임이 종료되지 않았을 때
            if state.check_first_player() == human_first:  # 사람이 둘 차례
                legal_actions = np.where(state.get_legal_actions() != 0)[0]
                my_action = input(f"Choose Your Action {tuple(legal_actions)}: ").strip()

                # ✅ 유효한 입력값만 허용
                while not my_action.isdigit() or int(my_action) not in legal_actions:
                    my_action = input(f"잘못된 입력입니다. {tuple(legal_actions)} 중에서 선택하세요: ").strip()

                my_action = int(my_action)
                state = state.next(my_action)
                print(f"Human ({'O' if human_first else 'X'}) chose: {my_action}")

            else:  # ✅ AlphaZero 차례
                action = self.get_action(state)
                state = state.next(action)
                print(f"AlphaZero ({'X' if human_first else 'O'}) chose: {action}")

            # ✅ 턴이 바뀔 때마다 보드 상태를 한 번만 출력
            self.render(state)

        print("Game Over!")  # 게임 종료 메시지 출력

    def __call__(self):
        self.vs_human()

In [ ]:
#정책 시각화와 현재 게임 상태를 그래프 형태로 표현
def visualize_history(history): #정책 값의 크기를 강조하고, 가장 높은 확률의 행동을 표시

    state, policy, _ = history #self-play 과정에서 기록된 하나의 상태 기록
    policy = np.array(policy).reshape(3,3) #알페제로가 생성한 확률 분포 -> 이를 3X3으로 변환!

    # Obtain is_first_player from the state object
    is_first_player = state.check_first_player() # state에서 is_first_player 정보를 얻어옵니다.


    def make_coord(state): #좌표 생성 함수 / 돌이 놓인 위치를 좌표로 변환하는 역할
        mask = state.astype(bool).reshape(-1)  # 불리언 마스크 생성
        indices = list(np.arange(len(mask))[mask])  # True인 인덱스 추출
        #state에서 돌이 놓인 위치를 True로 변환 후, 해당 좌표를 가져옴

        # 배열의 열 크기 (state는 2D 배열로 가정)
        width = state.shape[1]

        # divmod로 몫과 나머지를 계산
        if len(indices) != 0: #돌이 놓인 위치가 있을 경우
            quotients, remainders = zip(*[divmod(index, width) for index in indices])
            #divmod: 1D 인덱스를 2D 좌표 (row, col)로 변환
            coords = [quotients, remainders]

        else: #돌이 놓인 위치가 아닐 경우
            coords = None

        return coords


    # Plotting 현재 상태 시각화
    fig, ax = plt.subplots()
    cax = ax.matshow(policy, cmap='Greens', alpha=0.7)
    #policy 배열을 색상으로 표현 / 정책 확률 값의 크기를 초록색 음영으로 표현 / 색상 투명도


    # 플레이어의 차례 인식
    if is_first_player:  #첫번째 플레이어 차례 (즉, 알파제로)
        #각 플레이어의 돌 좌표 계산
        o_coords = make_coord(state.state) #state[0]: 첫번째 플레이어의 돌 좌표
        x_coords = make_coord(state.enemy_state) #state[1]: 두번째 플레이어의 돌 좌표

        #돌을 시각적으로 표시
        if o_coords is not None: #첫번째 플레이어 (O)의 돌이 보드에 있는 경우
            plt.scatter(o_coords[1], o_coords[0], marker="o", s=2000, c='black')  #큰 원 (검정색)
            plt.scatter(o_coords[1], o_coords[0], marker="o", s=1500, c='#F2F2F2') #작은 원 (흰색 내부)
            #플레이어의 돌을 시각적으로 표시
            #돌이 없을 때 불필요한 시각화 작업을 방지

        if x_coords is not None: #두번째 플레이어 (X)의 돌이 보드에 있는 경우
            plt.scatter(x_coords[1], x_coords[0], marker="x", s=2000, c='black', linewidths=3.0)


    else: #두번째 플레이어 차례 (즉, 인간)
        o_coords = make_coord(state.enemy_state) #state[1]: 두번째 플레이어의 돌 좌표
        x_coords = make_coord(state.state) #state[0]: 첫번째 플레이어의 돌 좌표

        #돌을 시각적으로 표시
        if o_coords is not None: #두번째 플레이어 (O)의 돌이 보드에 있는 경우
            plt.scatter(o_coords[1], o_coords[0], marker="o", s=2000, c='black')  # x, y 좌표로 매핑
            plt.scatter(o_coords[1], o_coords[0], marker="o", s=1500, c='#F2F2F2')

        if x_coords is not None:  #첫번째 플레이어 (X)의 돌이 보드에 있는 경우
            plt.scatter(x_coords[1], x_coords[0], marker="x", s=2000, c='black', linewidths=3.0)


    max_coord = divmod(np.argmax(policy), policy.shape[1]) #가장 높은 확률의 행동 강조
    #np.argmax(policy): 가장 높으 확률의 인덱스 추출
    #divmod: 1D 인덱스를 2D 좌표 (row, col)로 변환


    if is_first_player: #첫번째 플레이어 차례
        plt.scatter(max_coord[1], max_coord[0], marker="o", s=2000, c='red', linewidths=3.0) # #FF19A3
        #정책에서 가장 높은 확률의 행동을 빨간색으로 강조 -> 알파제로 차례일 때

    else: #두번째 플레이어 차례
        plt.scatter(max_coord[1], max_coord[0], marker="x", s=2000, c='red', linewidths=3.0) # #FF19A3
        #정책에서 가장 높은 확률의 행동을 빨간색으로 강조 -> 인간 차례일 때


    # Add text annotations (optional) / 정책 값 주석 추가
    for (i, j), value in np.ndenumerate(policy): #정책 배열의 각 위치와 값을 반환
        if value != 0.0:
            ax.text(j, i, f"{value:.2f}", ha='center', va='center', fontsize='15',
                    fontweight='bold' if value == np.max(policy) else "normal",
                    color="white" if value == np.max(policy) else "black")


    # Colorbar and titles / 색상 바와 타이틀 추가
    plt.colorbar(cax, ax=ax)
    ax.set_title("Current State, Policy with Highlighted Highest Action")
    ax.axis("off")
    plt.show()

In [ ]:
#1 모델 초기화 및 가중치 로드
model = Network(N_RESIDUAL_BLOCK, N_KERNEL, STATE_SIZE, N_ACTIONS)
model = nn.DataParallel(model).to(device)  # DataParallel로 병렬화

# self_play 객체 독립적으로 초기화
self_play = SelfPlay(model, SP_TEMPERATURE, SP_GAME_COUNT, EVAL_COUNT)

params = torch.load("/content/drive/MyDrive/3-2/강화/best_model_env_weights.pth", map_location=torch.device('cpu'), weights_only=False)

# Check if the checkpoint was saved using DataParallel by checking if keys have the 'module.' prefix
#The error occurs when the model is initialized directly with the saved weights without accounting for the 'module.' prefix if the model was saved using DataParallel.
#To fix this, you should modify the loading logic to add the 'module.' prefix to the keys of the saved checkpoint
if any(k.startswith('module.') for k in params.keys()):
    # If saved with DataParallel, load directly
    model.load_state_dict(params)
else:
    # If not saved with DataParallel, use the modified state_dict with prefix
    state_dict = {f"module.{k}": v for k, v in params.items()}  # Add 'module.' prefix
    model.load_state_dict(state_dict)

print("모델 가중치 로드 성공!")

모델 가중치 로드 성공!


사용자가 직접 숫자를 입력하면 현재 틱택토 게임의 보드 상태가 그림으로 나타난다.

- 플레이어 1 (알파제로 에이전트)의 말은 O이고, 플레이어 2 (인간)의 말은 X로 표현된다!

- 행동 확률에 따라 알파제로가 선택한 칸이 빨간색으로 강조되기도 한다.

In [ ]:
#2. 모델 대 인간 대결
# vs_human = ModelvsHuman(eval_network.best_model)

vs_human = ModelvsHuman(model) #객체를 초기화
vs_human() #__call__ 매서드를 통해 게임을 시작

선공(O)으로 플레이하려면 '1', 후공(X)으로 플레이하려면 '2'를 입력하세요: 1
  |   |  
  |   |  
  |   |  
-----------
Choose Your Action (0, 1, 2, 3, 4, 5, 6, 7, 8): 4
Human (O) chose: 4
  |   |  
  | O |  
  |   |  
-----------
AlphaZero (X) chose: 1
  | X |  
  | O |  
  |   |  
-----------
Choose Your Action (0, 2, 3, 5, 6, 7, 8): 0
Human (O) chose: 0
O | X |  
  | O |  
  |   |  
-----------
AlphaZero (X) chose: 3
O | X |  
X | O |  
  |   |  
-----------


In [ ]:
#2. self-play 기록 시각화
for idx in range(len(self_play.history)):
  visualize_history(self_play.history[idx])
#self_play.history: self-play에서 생성된 상태, 정책, 가치를 기록한 리스트
#self_play.history의 길이를 기반으로 인덱스를 생성하는 이터레이터 생성
#루프를 사용하여 기록 반복:

---------------

## 손실 (loss) 변화 분석

In [ ]:
import matplotlib.pyplot as plt

def plot_losses(loss_file="my_train_losses.csv"):
    if not os.path.exists(loss_file):
        print(f"{loss_file} 파일이 없습니다. 학습을 먼저 실행하세요.")
        return

    df = pd.read_csv(loss_file)

    plt.figure(figsize=(10, 5))
    plt.plot(df["epoch"], df["p_loss"], label="Policy Loss", color="blue")
    plt.plot(df["epoch"], df["v_loss"], label="Value Loss", color="red")
    plt.plot(df["epoch"], df["total_loss"], label="Total Loss", color="green")

    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training Loss Over Epochs")
    plt.legend()
    plt.grid()
    plt.show()

# 손실 그래프 출력
plot_losses()